In [ ]:
from importlib import import_module

ExtractOpenTabOutput = import_module('1-ExtractOpenTabOutput')
MergingBiz = import_module('2-MergingBiz')
RemoveInvalidCoordinates = import_module('3-RemoveInvalidCoordinates')
FilterNoAddrInfo = import_module('4-FilterNoAddrInfo')

# ExtractOpenTabOutput = __import__('1-ExtractOpenTabOutput')
# MergingBiz = __import__('2-MergingBiz')
# RemoveInvalidCoordinates = __import__('3-RemoveInvalidCoordinates')
# FilterNoAddrInfo = __import__('4-FilterNoAddrInfo')



In [7]:
ExtractOpenTabOutput.main()

Files unzipped to /home/jovyan/ODBiz/3-Merging/input


In [8]:
MergingBiz.main()

Begin merging files together...
100%|██████████| 60/60 [00:03<00:00, 16.15it/s]
Standardizing Province Names...
100%|██████████| 20/20 [00:05<00:00,  3.66it/s]
Standardizing Country Names...
100%|██████████| 1/1 [00:00<00:00,  1.66it/s]
Making Postal Codes consistent...
100%|██████████| 844131/844131 [00:00<00:00, 1935792.92it/s]
Marking Duplicates
Creating idx_basic...
100%|██████████| 844131/844131 [00:00<00:00, 2005533.08it/s]
Creating temp column:
Capitalize values and fill in blanks of dup_key columns with NULL to a copy of the df
100%|██████████| 9/9 [00:16<00:00,  1.84s/it]
Appending dup_key columns together and assign to temp column
100%|██████████| 9/9 [00:02<00:00,  3.99it/s]
Applying hashing to get new indicies...
100%|██████████| 844131/844131 [00:00<00:00, 1970954.60it/s]
Filling in geo_source column
Dropping duplicates
41567 duplicate rows dropped
Writing 802564 dataframe entries to csv. This will take a while and unfortunately no easy progress bar solutions were availabl

In [11]:
RemoveInvalidCoordinates.main()

Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.

In [10]:
FilterNoAddrInfo.main()

Loading data: 100%|██████████| 13/13 [00:07<00:00,  1.75it/s]
Filling in NA as empty string
Done in 7 s
Concatenating address info
Done in 13 s
Checking for blanks
Done in 0 s
Dropping entries with insufficient address info
Done in 2 s
Saving df to /home/jovyan/ODBiz/3-Merging/output/4-ODBiz_merged_2022-07-19.csv
Done in 20 s
df saved to /home/jovyan/ODBiz/3-Merging/output/4-ODBiz_merged_2022-07-19.csv
